# Analysis of bitcoin's additional risk metrics

## Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# Set charts theme
sns.set_theme(style="darkgrid", rc={"grid.alpha": 0.33})
plt.style.use("dark_background")

# Save chart as png function
def save_chart_as_png(filename: str) -> None:
    plt.savefig(
        f"../images/{filename}.png",
        format="png",
        dpi=300,
        orientation="landscape",
        bbox_inches="tight",
    )

In [ ]:
# Get all dfs
def get_df(csv_basename: str) -> pd.DataFrame:
    # Get df from CSV with date as index
    return pd.read_csv(f"../data/{csv_basename}.csv", index_col="date", parse_dates=True)

df_btc = get_df("BTC")
df_us10y = get_df("US10Y")

## Yearly risk-adjusted returns across time ⚖️

### Sharpe ratio

In [ ]:
# Get YoY returns
# Get yearly bitcoin price df with first and last prices
df_btc_yearly = df_btc.groupby(df_btc.index.year)["price"].agg(
    first_price="first",
    last_price="last",
)
# Get YoY return
df_btc_yearly["price_change"] = (df_btc_yearly["last_price"] - df_btc_yearly["first_price"]) / df_btc_yearly["first_price"]

In [ ]:
# Get the volatility of monthly returns along with the number of months on yearly df
monthly_returns = df_btc["price"].resample("ME").ffill().pct_change() # Resample to monthly (month end)
monthly_stats = monthly_returns.groupby(monthly_returns.index.year).agg(
    volatility_m=("std"),
    num_months=("count"),
)
df_btc_yearly[["volatility_m", "num_months"]] = monthly_stats.reindex(df_btc_yearly.index)

In [ ]:
# Get risk free rate for each year from the 10-year US treasury yield average
df_btc_yearly["us10_yield_avg"] = df_us10y["yield"].groupby(df_us10y.index.year).mean()

In [ ]:
# Calculate Sharpe ratio, except for incomplete years (2010 and 2024)
df_btc_yearly.loc[df_btc_yearly["num_months"] == 12, "sharpe_ratio"] = round((df_btc_yearly["price_change"] - df_btc_yearly["us10_yield_avg"]) / df_btc_yearly["volatility_m"], 3)

In [ ]:
plt.figure(figsize=(10, 6))

sns.barplot(data=df_btc_yearly[df_btc_yearly["sharpe_ratio"].notnull()], x="date", y="sharpe_ratio", color="lightblue")

plt.title("Yearly Sharpe ratio of bitcoin across time")
plt.xlabel("")
plt.ylabel("")

plt.yticks([-15, -5, 0, 5, 10, 40])

save_chart_as_png("3.2_BTC_yearly_sharpe")

In [ ]:
# Highest yearly Sharpe ratio
df_btc_yearly.loc[[df_btc_yearly["sharpe_ratio"].idxmax()]]

In [ ]:
# Lowest yearly Sharpe ratio
df_btc_yearly.loc[[df_btc_yearly["sharpe_ratio"].idxmin()]]

In [ ]:
# Create table with yearly Sharpe ratios stats
pd.DataFrame({
    "Average yearly Sharpe ratio": [round(df_btc_yearly["sharpe_ratio"].mean(), 4)],
    "Median yearly Sharpe ratio": [round(df_btc_yearly["sharpe_ratio"].median(), 4)],
    "Standard deviation": [round(df_btc_yearly["sharpe_ratio"].std(), 4)],
    "Min yearly Sharpe ratio": [round(df_btc_yearly["sharpe_ratio"].min(), 4)],
    "Max yearly Sharpe ratio": [round(df_btc_yearly["sharpe_ratio"].max(), 4)],
})

**Key takeaways:**
- ...

### Sortino ratio

In [ ]:
# Get the volatility of monthly negative returns on yearly df
downside_monthly_returns = monthly_returns[monthly_returns < 0]
downside_monthly_volatility = downside_monthly_returns.groupby(downside_monthly_returns.index.year).std()
df_btc_yearly["downside_volatility_m"] = downside_monthly_volatility.reindex(df_btc_yearly.index)

In [ ]:
# Calculate Sortino ratio, except for incomplete years (2010 and 2024)
df_btc_yearly.loc[df_btc_yearly["num_months"] == 12, "sortino_ratio"] = round((df_btc_yearly["price_change"] - df_btc_yearly["us10_yield_avg"]) / df_btc_yearly["downside_volatility_m"], 3)

In [ ]:
plt.figure(figsize=(10, 6))

sns.barplot(data=df_btc_yearly[df_btc_yearly["sortino_ratio"].notnull()], x="date", y="sortino_ratio", color="lime")

plt.title("Yearly Sortino ratio of bitcoin across time")
plt.xlabel("")
plt.ylabel("")

#plt.yscale("symlog")

plt.yticks([-100, -25, 0, 25, 100, 200, 350])

save_chart_as_png("3.2_BTC_yearly_sortino")

In [ ]:
# Highest yearly Sortino ratio
df_btc_yearly.loc[[df_btc_yearly["sortino_ratio"].idxmax()]]

In [ ]:
# Lowest yearly Sortino ratio
df_btc_yearly.loc[[df_btc_yearly["sortino_ratio"].idxmin()]]

In [ ]:
# Create table with yearly Sortino ratios stats
pd.DataFrame({
    "Average yearly Sortino ratio": [round(df_btc_yearly["sortino_ratio"].mean(), 4)],
    "Median yearly Sortino ratio": [round(df_btc_yearly["sortino_ratio"].median(), 4)],
    "Standard deviation": [round(df_btc_yearly["sortino_ratio"].std(), 4)],
    "Min yearly Sortino ratio": [round(df_btc_yearly["sortino_ratio"].min(), 4)],
    "Max yearly Sortino ratio": [round(df_btc_yearly["sortino_ratio"].max(), 4)],
})

**Key takeaways:**
- ...

## Value at risk (VaR) 🚨

### Historical method

In [ ]:
# Get log price change
df_btc["price_change_log"] = np.log(df_btc["price"] / df_btc["price"].shift(1))

In [ ]:
# Create a table with 95% and 99% confidence interval VaR for different time horizons
def calculate_hist_var(time_horizon_days: int, confidence_interval: float) -> float:
    # Aggregate log returns over the specified time horizon
    aggregated_returns = df_btc["price_change_log"].rolling(window=time_horizon_days).sum().dropna()

    # Convert confidence interval to corresponding percentile
    percentile = (1 - confidence_interval) * 100

    # Calculate the historical VaR as the value at the specified percentile of aggregated returns
    return -np.percentile(aggregated_returns, percentile).round(2)

confidence_intervals = [0.95, 0.99]

pd.DataFrame({
    "Confidence Interval": confidence_intervals,
    "1-month VaR": [calculate_hist_var(30, ci) for ci in confidence_intervals],
    "1-quarter VaR": [calculate_hist_var(90, ci) for ci in confidence_intervals],
    "1-year VaR": [calculate_hist_var(365, ci) for ci in confidence_intervals],
})

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 6), sharey=True)

time_horizons_days = [30, 90, 365]
colors = ["turquoise", "purple", "green"]
titles = ["30-day historical returns", "90-day historical returns", "365-day historical returns"]

for i, time_horizon_days in enumerate(time_horizons_days):
    aggregated_returns = df_btc["price_change_log"].rolling(window=time_horizon_days).sum().dropna()
    
    sns.histplot(aggregated_returns, stat="probability", binwidth=0.1, binrange=(-1, 1), color=colors[i], edgecolor="white", alpha=0.75, ax=axes[i])
    axes[i].axvline(np.percentile(aggregated_returns, 5), color="orange", linewidth=1.5, linestyle="--", label="VaR at 95% confidence level")
    axes[i].axvline(np.percentile(aggregated_returns, 1), color="red", linewidth=1.5, linestyle="--", label="VaR at 99% confidence level")
    
    axes[i].set_title(f"Distribution of the {titles[i]}")
    axes[i].set_xlabel("")
    
    axes[i].set_xlim(-1, 1)

axes[0].set_ylabel("Probability")
axes[2].legend(loc="upper right")

plt.tight_layout()

**Key takeaways:**
- ...

### Monte Carlo method

In [ ]:
# Create a table with 95% and 99% confidence interval VaR for different time horizons
def calculate_monte_carlo_var(mean: float, std: float, num_simulations: int, time_horizon_days: int, confidence_interval: float) -> float:
    # Simulate future returns using a normal distribution (output is array of x days by y simulations)
    simulated_returns = np.random.normal(mean, std, (num_simulations, time_horizon_days))
    
    # Aggregate returns over the time horizon (sum x days of each simulation)
    aggregated_returns = simulated_returns.sum(axis=1)

    # Convert confidence interval to corresponding percentile
    percentile = (1 - confidence_interval) * 100

    # Calculate VaR as the value at the specified percentile of simulated returns
    return -np.percentile(aggregated_returns, percentile).round(2)
    
mean = df_btc["price_change_log"].mean()
std = df_btc["price_change_log"].std()
num_simulations = 10_000
confidence_intervals = [0.95, 0.99]

pd.DataFrame({
    "Confidence Interval": confidence_intervals,
    "1-month VaR": [calculate_monte_carlo_var(mean, std, num_simulations, 30, ci) for ci in confidence_intervals],
    "1-quarter VaR": [calculate_monte_carlo_var(mean, std, num_simulations, 90, ci) for ci in confidence_intervals],
    "1-year VaR": [calculate_monte_carlo_var(mean, std, num_simulations, 365, ci) for ci in confidence_intervals],
})

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 6), sharey=True)

mean = df_btc["price_change_log"].mean()
std = df_btc["price_change_log"].std()
num_simulations = 10_000
time_horizons_days = [30, 90, 365]
colors = ["turquoise", "purple", "green"]
titles = ["30-day simulated returns", "90-day simulated returns", "365-day simulated returns"]

for i, time_horizon_days in enumerate(time_horizons_days):
    simulated_returns = np.random.normal(mean, std, (num_simulations, time_horizon_days))  
    aggregated_returns = simulated_returns.sum(axis=1)
    
    sns.histplot(aggregated_returns, stat="probability", binwidth=0.1, binrange=(-1, 1), color=colors[i], edgecolor="white", alpha=0.75, ax=axes[i])
    axes[i].axvline(np.percentile(aggregated_returns, 5), color="orange", linewidth=1.5, linestyle="--", label="VaR at 95% confidence level")
    axes[i].axvline(np.percentile(aggregated_returns, 1), color="red", linewidth=1.5, linestyle="--", label="VaR at 99% confidence level")
    
    axes[i].set_title(f"Distribution of the {titles[i]}")
    axes[i].set_xlabel("")
    
    axes[i].set_xlim(-1, 1)

axes[0].set_ylabel("Probability")
axes[2].legend(loc="upper right")

plt.tight_layout()

**Key takeaways:**
- ...

### Conditional value at risk (CVaR) (or expected shortfall)

In [ ]:
# Create a table with 95% and 99% confidence interval CVaR for different time horizons
def calculate_monte_carlo_cvar(mean: float, std: float, num_simulations: int, time_horizon_days: int, confidence_interval: float) -> float:
    # Simulate future returns using a normal distribution (output is array of x days by y simulations)
    simulated_returns = np.random.normal(mean, std, (num_simulations, time_horizon_days))
    
    # Aggregate returns over the time horizon (sum x days of each simulation)
    aggregated_returns = simulated_returns.sum(axis=1)

    # Convert confidence interval to corresponding percentile
    percentile = (1 - confidence_interval) * 100

    # Calculate VaR as the value at the specified percentile of simulated returns
    var = np.percentile(aggregated_returns, percentile).round(2)

    # Calculate CVaR as the mean of all returns below the VaR threshold
    return -aggregated_returns[aggregated_returns <= var].mean()
    
mean = df_btc["price_change_log"].mean()
std = df_btc["price_change_log"].std()
num_simulations = 10_000
confidence_intervals = [0.95, 0.99]

pd.DataFrame({
    "Confidence Interval": confidence_intervals,
    "1-month CVaR": [calculate_monte_carlo_cvar(mean, std, num_simulations, 30, ci) for ci in confidence_intervals],
    "1-quarter CVaR": [calculate_monte_carlo_cvar(mean, std, num_simulations, 90, ci) for ci in confidence_intervals],
    "1-year CVaR": [calculate_monte_carlo_cvar(mean, std, num_simulations, 365, ci) for ci in confidence_intervals],
})

**Key takeaways:**
- ...